1. Import LLM
2. Import tool
3. Bind to LLM
4. Prompt template
5. custom agent with memory 
6. Showcase in gradio

Import LLM

In [1]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="gpt-3.5-turbo", 
                 temperature=0, 
                 openai_api_key=os.environ.get("OPENAI_API_KEY"))

Import Tool

In [2]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [4]:
wikipedia.run("Highest goals in a single season of La Liga")

"Page: La Liga records and statistics\nSummary: The La Liga is a Spanish professional league for association football club. At the top of the Spanish football league system, it is the country's primary football competition and is contested by 20 clubs. The competition was formed in 1929, with an initial format of 10 teams.\n\nPage: List of La Liga top scorers\nSummary: La Liga all-time top scorer is Lionel Messi with 474 goals, all for Barcelona. He also holds the record for most goals scored in a single season with 50 in the 2011–12 campaign, and is the only player ever to win the league's top scorer award in eight different seasons. Athletic Bilbao's Telmo Zarra, who was the competition's all-time top scorer for sixty years until 2014, won the top scorer award six times. Three other players — Real Madrid's Alfredo Di Stéfano, Quini of Sporting Gijón and Barcelona, and Hugo Sánchez of Atlético Madrid and Real Madrid — each finished as top scorer in five individual seasons. \nAlfredo D

Bind Tool with LLM

In [5]:
tools = [wikipedia]

In [6]:
llm_with_tools = llm.bind_tools(tools)

Create Prompt for LLM

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [8]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

Create Custom Agent with Memory

In [9]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser


agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [10]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, stream_runnable = False)

In [11]:
from langchain.callbacks import get_openai_callback

In [12]:
with get_openai_callback() as cb:
    out = agent_executor.invoke({"input": "Highest goals in a single season of La Liga", "chat_history": chat_history})
print(out)
print(cb)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `La Liga highest goals in a single season`


Page: La Liga records and statistics
Summary: The La Liga is a Spanish professional league for association football club. At the top of the Spanish football league system, it is the country's primary football competition and is contested by 20 clubs. The competition was formed in 1929, with an initial format of 10 teams.

Page: List of La Liga top scorers
Summary: La Liga all-time top scorer is Lionel Messi with 474 goals, all for Barcelona. He also holds the record for most goals scored in a single season with 50 in the 2011–12 campaign, and is the only player ever to win the league's top scorer award in eight different seasons. Athletic Bilbao's Telmo Zarra, who was the competition's all-time top scorer for sixty years until 2014, won the top scorer award six times. Three other players — Real Madrid's Alfredo Di Stéfano, Quini of Sporting Gijón and Barcelona, and Hugo Sánc

In [13]:
chat_history.extend(
    [
        HumanMessage(content="Highest goals in a single season of La Liga"),
        AIMessage(content=out["output"]),
    ]
)

print(chat_history)
agent_executor.invoke({"input": "How many goals he has scored overall in L Liga?", "chat_history": chat_history})

[HumanMessage(content='Highest goals in a single season of La Liga'), AIMessage(content='The record for the most goals scored in a single season in La Liga is held by Lionel Messi with 50 goals in the 2011-2012 campaign.')]


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Lionel Messi`


Page: Lionel Messi
Summary: Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d'Or awards, a record six European Golden Shoes, and was named the world's best player for a record eight times by FIFA. Until 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles, and the UEFA Champions Leag

{'input': 'How many goals he has scored overall in L Liga?',
 'chat_history': [HumanMessage(content='Highest goals in a single season of La Liga'),
  AIMessage(content='The record for the most goals scored in a single season in La Liga is held by Lionel Messi with 50 goals in the 2011-2012 campaign.')],
 'output': 'Lionel Messi has scored a total of 474 goals in La Liga.'}

Showcase in Gradio

In [14]:
import gradio as gr

In [15]:
agent_history = []
def call_agent(query, chat_history):
    print("Chat history : ", chat_history)
    output = agent_executor.invoke({"input": query, "chat_history": agent_history})

    agent_history.extend(
    [
        HumanMessage(content="Highest goals in a single season of La Liga"),
        AIMessage(content=out["output"]),
    ]
    )


    chat_history += [
        [
            "<b>Question: </b>" + query,
            "<b>Answer: </b>" + output["output"]
        ]
    ]


    return output["output"], chat_history

In [16]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label = "QnA with Wikipedia")
    question = gr.Textbox(label = "Ask you query here")

    with gr.Row():
        submit = gr.Button("Submit")
        clear = gr.ClearButton([chatbot, question])

    def user(user_message, history):

        bot_message, history = call_agent(user_message, history)

        return "", history

    question.submit(user, [question, chatbot], [question, chatbot], queue=False)
    submit.click(user, [question, chatbot], [question, chatbot], queue=False)
    
demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.38.0, however version 4.29.0 is available, please upgrade.
--------
Chat history :  []


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Most goals scored in a single season of La Liga`


Page: List of La Liga top scorers
Summary: La Liga all-time top scorer is Lionel Messi with 474 goals, all for Barcelona. He also holds the record for most goals scored in a single season with 50 in the 2011–12 campaign, and is the only player ever to win the league's top scorer award in eight different seasons. Athletic Bilbao's Telmo Zarra, who was the competition's all-time top scorer for sixty years until 2014, won the top scorer award six times. Three other players — Real Madrid's Alfredo Di Stéfano, Quini of Sporting Gijón and Barcelona, and Hugo Sánchez of Atlético Madrid and Real Madrid — each finished as top scorer in five individual seasons. 
Alfredo Di Stéfano was the first non-European player to score a hundred La Liga goals, tho